In [ ]:
# Assuming rate_map, invoice_df, and generate_cost_curve are already defined
# Also assumes export_freight_chart_excel is defined (from earlier cell)
import os

output_dir = "./freight_output"
os.makedirs(output_dir, exist_ok=True)

for site in invoice_df['site'].unique():
    site_data = invoice_df[invoice_df['site'] == site]
    for commodity in site_data['commodity_group'].unique():
        curve_df = generate_cost_curve(rate_map, site, commodity)
        merge_and_export(curve_df, invoice_df, site, commodity, output_dir)

        # Merge vendor + invoice data for charting
        invoice_subset = invoice_df[(invoice_df['site'] == site) & (invoice_df['commodity_group'] == commodity)].copy()
        invoice_subset = invoice_subset.rename(columns={
            'market_freight_costs': 'actual_freight_cost'
        })
        plot_ready_df = pd.concat([curve_df, invoice_subset], ignore_index=True)

        # Create Excel chart output
        excel_chart_file = os.path.join(output_dir, f"{site}_{commodity}_freight_costs_with_chart.xlsx")
        export_freight_chart_excel(plot_ready_df, site, commodity, excel_chart_file)

        # Optional: Plot inline as visual confirmation
        plot_cost_curve(curve_df, invoice_df, site, commodity)